In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import simpy
from LEOEnvironmentRL import initialize, load_route_from_csv  # Use RL version
import pandas as pd
import os
from stable_baselines3 import DQN
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
import torch
import random

# %% 
import sb3_contrib
from HandoverEnvironment import LEOEnv as LEOEnvPPO 
from HandoverEnvironment import mask_fn, predict_valid_action
from HandoverEnvironment_DQN import LEOEnv as LEOEnvDQN
from HandoverEnvironment_DQN import predict_valid_action as predict_valid_action_dqn
from HandoverEnvironment_ODT import LEOEnv as LEOEnvODT
from HandoverEnvironment_ODT import predict_valid_action_dt
from ODT import OnlineDecisionTransformer
from LEOEnvironment import LEOEnv as LEOEnvBase

['ARS', 'CrossQ', 'MaskablePPO', 'QRDQN', 'RecurrentPPO', 'TQC', 'TRPO', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'ars', 'common', 'crossq', 'file_handler', 'os', 'ppo_mask', 'ppo_recurrent', 'qrdqn', 'tqc', 'trpo', 'version_file']


In [ ]:
# Load observations from csv file 
obs_base_list = pd.read_csv('observations_base.csv', header=None).values.tolist()
obs_dqn_list = pd.read_csv('observations_dqn.csv', header=None).values.tolist()
obs_ppo_list = pd.read_csv('observations_ppo.csv', header=None).values.tolist()
obs_odt_list = pd.read_csv('observations_odt.csv', header=None).values.tolist()

In [ ]:
obs_base_list = np.array(obs_base_list)
obs_dqn_list = np.array(obs_dqn_list)    
obs_ppo_list = np.array(obs_ppo_list)
obs_odt_list = np.array(obs_odt_list)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(obs_base_list[:, 6], label = 'baseline handovers')
plt.plot(obs_ppo_list[:, 6], label = 'ppo handovers')
plt.plot(obs_dqn_list[:, 6], label = 'dqn handovers')
plt.plot(obs_odt_list[:, 6], label = 'odt handovers')
plt.legend()
plt.title('Total number of handovers across runs')
plt.show()

In [ ]:
plt.plot(obs_base_list[:, 7], label = 'baseline allocated bandwidth')
plt.plot(obs_ppo_list[:, 7], label = 'ppo allocated bandwidth')
plt.plot(obs_dqn_list[:, 7], label = 'dqn allocated bandwidth')
plt.plot(obs_odt_list[:, 7], label = 'odt allocated bandwidth')
plt.legend()
plt.title('Total allocated bandwidth across runs')
plt.show()

In [ ]:
# Get average allocation to demand across runs 
avg_allocated_to_demand_base = []
avg_allocated_to_demand_ppo = []
avg_allocated_to_demand_dqn = []
avg_allocated_to_demand_odt = []

for i in range(len(obs_base_list)): 
    avg_allocated_to_demand_base.append(np.mean(obs_base_list[:i, 8]))
    avg_allocated_to_demand_ppo.append(np.mean(obs_ppo_list[:i, 8]))
    avg_allocated_to_demand_dqn.append(np.mean(obs_dqn_list[:i, 8]))
    avg_allocated_to_demand_odt.append(np.mean(obs_odt_list[:i, 8]))

In [ ]:
plt.plot(avg_allocated_to_demand_base, label = 'baseline average allocation to demand')
plt.plot(avg_allocated_to_demand_ppo, label = 'ppo average allocation to demand')
plt.plot(avg_allocated_to_demand_dqn, label = 'dqn average allocation to demand')
plt.plot(avg_allocated_to_demand_odt, label = 'odt average allocation to demand')
plt.legend()
plt.title('Average Allocation to Demand Across Runs')
plt.show()

In [ ]:
# Get average allocation to demand across runs 
allocated_to_demand_base = []
allocated_to_demand_ppo = []
allocated_to_demand_dqn = []
allocated_to_demand_odt = []

for i in range(len(obs_base_list)): 
    allocated_to_demand_base.append(obs_base_list[i, 8])
    allocated_to_demand_ppo.append(obs_ppo_list[i, 8])
    allocated_to_demand_dqn.append(obs_dqn_list[i, 8])
    allocated_to_demand_odt.append(obs_odt_list[i, 8])

In [ ]:
plt.plot(allocated_to_demand_base, label = 'baseline average allocation to demand')
plt.plot(allocated_to_demand_ppo, label = 'ppo average allocation to demand')
plt.plot(allocated_to_demand_dqn, label = 'dqn average allocation to demand')
plt.plot(allocated_to_demand_odt, label = 'odt average allocation to demand')
plt.legend()
plt.title('Average Allocation to Demand Across Runs')
plt.show()